In [1]:
# Importing modules
import numpy as np
import pandas as pd
import numba
import pyarrow.dataset as ds

In [2]:
# Pandas display options (personal preference)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

In [3]:
# Importing the G09 parquet file
dataset = ds.dataset("../../data/WD02/WD02_2p4.parquet", format="parquet")
table = dataset.to_table()

# Setting uberID as index
df = table.to_pandas().set_index('uberID')
df.index=df.index.astype('int64')

del dataset,table

In [4]:
# Getting rid of weird spaces in the class column
df=df.replace({'class': {'ambiguous ': 'ambiguous',
                         'star      ': 'star', 
                         'galaxy    ': 'galaxy', 
                         'artefact  ':'artefact'}})

In [5]:
# Making a number of cuts: magnitude, mask, starmask and removing artefacts
df=df[(df['mag_Zt']<22) & 
      (df['mask']==0) & 
      (df['starmask']==0) & 
      (df['class']!='artefact')&
      (df['duplicate']==0)
]

In [6]:
# Getting a 1/10 subset of the data (for the purpose of this tutorial)
# df=df[::10]

In [7]:
df

,FrameName,FrameID,segID,xmax,ymax,censep,RAcen,Deccen,RAmax,Decmax,RAGAIA_r,RAGAIA,DecGAIA,DecGAIA_r,RAGAIA_r_cen,DecGAIA_r_cen,RAcen_gt,Deccen_gt,RAmax_gt,Decmax_gt,RAcen_rt,Deccen_rt,RAmax_rt,Decmax_rt,RAcen_Zt,Deccen_Zt,RAmax_Zt,Decmax_Zt,sky_mean,skyRMS_mean,log10seeing,log10seeing_r,log10seeing_i,log10seeing_Z,log10seeing_Y,mag,EBV,R50,R50_gt,R50_rt,R50_Yt,R50_Jt,R50_Ht,R90,R100,N100,axrat,ang,groupID,Ngroup,mag_app_Zt,mag_Zt,mag_app_gt,mag_app_rt,mag_app_it,mag_app_Yt,flux_ut,flux_err_ut,flux_ut_uncorrected,flux_err_ut_uncorrected,flux_gt,flux_err_gt,flux_gt_uncorrected,flux_err_gt_uncorrected,flux_rt,flux_err_rt,flux_rt_uncorrected,flux_err_rt_uncorrected,flux_it,flux_err_it,flux_it_uncorrected,flux_err_it_uncorrected,flux_Zt,flux_err_Zt,flux_Zt_uncorrected,flux_err_Zt_uncorrected,flux_Yt,flux_err_Yt,flux_Yt_uncorrected,flux_err_Yt_uncorrected,flux_Jt,flux_err_Jt,flux_Jt_uncorrected,flux_err_Jt_uncorrected,flux_Ht,flux_err_Ht,flux_Ht_uncorrected,flux_err_Ht_uncorrected,flux_Kt,flux_err_Kt,flux_Kt_uncorrected,flux_err_Kt_uncorrected,flux_W1t,flux_err_W1t,flux_W1t_uncorrected,flux_err_W1t_uncorrected,flux_W2t,flux_err_W2t,flux_W2t_uncorrected,flux_err_W2t_uncorrected,flux_uc,flux_err_uc,flux_gc,flux_err_gc,flux_rc,flux_err_rc,flux_ic,flux_err_ic,flux_Zc,flux_err_Zc,flux_Yc,flux_err_Yc,flux_Jc,flux_err_Jc,flux_Hc,flux_err_Hc,flux_Kc,flux_err_Kc,flux_W1c,flux_err_W1c,flux_W2c,flux_err_W2c,mask,starmask,starscol,starssize,class,noOPT_r,noOPT_i,noIR_Z,noIR_Y,duplicate,mag_Zc,mag_Zt_fake
uberID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
36971195800254,waves_postprocessed_37.0_-3.5.rds,3697.0,28,11957.5,253.5,8276.934819,36.585878,-4.062106,36.585862,-4.062085,36.585896,36.585896,-4.062111,-4.062111,36.585931,-4.062128,36.585918,-4.062081,36.585862,-4.062085,36.585897,-4.062102,36.585862,-4.062085,36.585880,-4.062105,36.585862,-4.062085,-0.002751,0.003172,-0.162571,-0.094759,-0.164494,-0.161807,-0.153093,15.489770,0.033257,0.670359,0.980934,0.849038,0.661603,0.601284,0.565228,2.173057,11.660159,4442,0.935971,73.598735,28,10,16.045604,15.467905,18.671152,17.160600,16.335561,15.793864,3.380947e-05,2.207446e-06,2.917651e-05,1.904956e-06,2.949221e-04,8.025781e-07,2.636068e-04,7.173589e-07,1.028585e-03,9.746135e-07,9.482575e-04,8.985013e-07,1.729963e-03,3.315048e-06,1.623443e-03,3.110929e-06,0.002360,2.875208e-06,0.002250,2.741703e-06,0.002961,6.667548e-06,0.002853,6.424377e-06,0.002982,3.570833e-06,0.002903,3.476267e-06,0.003652,3.577042e-06,0.003589,3.515583e-06,0.002888,5.177355e-06,0.002855,5.119185e-06,1.286970e-03,8.837989e-07,1.279061e-03,8.783678e-07,6.698090e-04,3.140175e-06,6.669475e-04,3.126760e-06,2.929688e-05,1.597023e-06,2.939962e-04,5.770235e-07,1.022830e-03,6.629288e-07,1.721307e-03,1.980520e-06,2.350237e-03,1.182835e-06,2.947524e-03,2.627846e-06,0.002970,1.530534e-06,3.634016e-03,1.619906e-06,0.002868,2.629908e-06,1.218787e-03,3.587485e-07,6.390815e-04,1.514522e-06,0.0,0.0,3.0,3.0,star,0.0,0.0,0.0,0.0,0.0,15.472221,0
36971349600916,waves_postprocessed_37.0_-3.5.rds,3697.0,199,13495.5,915.5,9009.575018,36.457390,-4.006810,36.457419,-4.006849,36.457371,36.457454,-4.006875,-4.006875,36.457419,-4.006844,36.457410,-4.006829,36.457419,-4.006849,36.457384,-4.006818,36.457336,-4.006849,36.457388,-4.006811,36.457419,-4.006849,-0.001396,0.005437,-0.162571,-0.094759,-0.164494,-0.161807,-0.153093,15.037637,0.039474,3.169236,3.589942,3.428132,3.026679,2.869277,2.729232,8.409229,17.956163,9260,0.822766,158.393736,199,15,17.409777,15.003946,19.049951,18.164815,17.612484,17.025651,2.486257e-04,3.700196e-06,2.087260e-04,3.106384e-06,1.168059e-03,2.497155e-06,1.022353e-03,2.185656e-06,2.226178e-03,3.665841e-06,2.021366e-03,3.328578e-06,2.930788e-03,5.943551e-06,2.717851e-03,5.511720e-06,0.003618,7.792429e-06,0.003419,7.364852e-06,0.004738,1.878522e-05,0.004534,1.797484e-05,0.005026,1.668712e-05,0.004869,1.616390e-05,0.006256,1.927160e-05,0.006

In [8]:
# Reading the u and no u labels from notebooks 1 and 2, and creating new dataframes
df_u=pd.read_csv('../../data/WD02/WD02_u.csv').set_index('uberID')
df_no_u=pd.read_csv('../../data/WD02/WD02_no_u.csv').set_index('uberID')

In [9]:
# Creating new columns in the main dataframe with u and no u labels
df['cluster_label']=np.zeros(len(df))
df['cluster_label_no_u']=np.zeros(len(df))

In [10]:
# Creating umap columns in the main dataframe with u and no u labels
for i in range(10):
    df[f'umap{i+1}_no_u'] = np.zeros(len(df))
    df[f'umap{i+1}_u'] = np.zeros(len(df))

In [11]:
# Filling the new columns with u and no u labels
df.loc[df_u.index,'cluster_label']=df_u['cluster_label'].apply(str)
df.loc[df_no_u.index,'cluster_label_no_u']=df_no_u['cluster_label'].apply(str)

/tmp/ipykernel_275672/2692133000.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['star' 'galaxy' 'star' ... 'galaxy' 'galaxy' 'galaxy']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df_u.index,'cluster_label']=df_u['cluster_label'].apply(str)
/tmp/ipykernel_275672/2692133000.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['star' 'galaxy' 'star' ... 'galaxy' 'galaxy' 'galaxy']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df_no_u.index,'cluster_label_no_u']=df_no_u['cluster_label'].apply(str)


In [12]:
for i in range(10):
    df.loc[df_u.index,f'umap{i+1}_u']=df_u[f'umap{i+1}']
    df.loc[df_no_u.index,f'umap{i+1}_no_u']=df_no_u[f'umap{i+1}']

In [13]:
labels=['star','galaxy']

In [14]:
# Creating a 'final label' column full of 1's to combine the two labels
df['final_label']=np.ones(len(df))


In [15]:
# Creating umap columns with 1's
for i in range(10):
    df[f'umap{i+1}'] = np.ones(len(df))


In [16]:
# First applying the no u labels to the final label
for label in labels:
    df.loc[df[df['cluster_label_no_u']==label].index,'final_label']=label
    # First putting umap columns with no u labels
    for i in range(10):
        df.loc[df[df['cluster_label_no_u']==label].index, f'umap{i+1}'] = df.loc[df[df['cluster_label_no_u']==label].index, f'umap{i+1}_no_u']

/tmp/ipykernel_275672/1789200346.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'star' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df[df['cluster_label_no_u']==label].index,'final_label']=label


In [17]:
# Then applying the u labels to the final label, overwriting any existing no u labels
for label in labels:
    df.loc[df[df['cluster_label']==label].index,'final_label']=label

    for i in range(10):
        df.loc[df[df['cluster_label']==label].index, f'umap{i+1}'] = df.loc[df[df['cluster_label']==label].index, f'umap{i+1}_u']

In [18]:
# Convering final label to string
df['final_label']=df['final_label'].apply(str)

In [19]:
# Creating a new column to indicate whether a source has any missing bands.
# If final label still == 1, that indicates a label hasn't been added, so it has a missing band
df['missing_bands']=True
df.loc[df[df['final_label']!='1.0'].index,'missing_bands']=False

In [20]:
# For the sources with missing labels we then overwrite with class
df.loc[df[df['final_label']=='1.0'].index,'final_label'] = df[df['final_label']=='1.0']['class']

In [21]:
# For sources with missing labels we put nan in the umap columns
for i in range(10):
    df.loc[df[df['missing_bands']==True].index, f'umap{i+1}'] = np.nan

In [22]:
# Sources with missing bands are only 1.5% of the sample
len(df[df['missing_bands']==True])/len(df)

0.029863920559029054

In [23]:
df

,FrameName,FrameID,segID,xmax,ymax,censep,RAcen,Deccen,RAmax,Decmax,RAGAIA_r,RAGAIA,DecGAIA,DecGAIA_r,RAGAIA_r_cen,DecGAIA_r_cen,RAcen_gt,Deccen_gt,RAmax_gt,Decmax_gt,RAcen_rt,Deccen_rt,RAmax_rt,Decmax_rt,RAcen_Zt,Deccen_Zt,RAmax_Zt,Decmax_Zt,sky_mean,skyRMS_mean,log10seeing,log10seeing_r,log10seeing_i,log10seeing_Z,log10seeing_Y,mag,EBV,R50,R50_gt,R50_rt,R50_Yt,R50_Jt,R50_Ht,R90,R100,N100,axrat,ang,groupID,Ngroup,mag_app_Zt,mag_Zt,mag_app_gt,mag_app_rt,mag_app_it,mag_app_Yt,flux_ut,flux_err_ut,flux_ut_uncorrected,flux_err_ut_uncorrected,flux_gt,flux_err_gt,flux_gt_uncorrected,flux_err_gt_uncorrected,flux_rt,flux_err_rt,flux_rt_uncorrected,flux_err_rt_uncorrected,flux_it,flux_err_it,flux_it_uncorrected,flux_err_it_uncorrected,flux_Zt,flux_err_Zt,flux_Zt_uncorrected,flux_err_Zt_uncorrected,flux_Yt,flux_err_Yt,flux_Yt_uncorrected,flux_err_Yt_uncorrected,flux_Jt,flux_err_Jt,flux_Jt_uncorrected,flux_err_Jt_uncorrected,flux_Ht,flux_err_Ht,flux_Ht_uncorrected,flux_err_Ht_uncorrected,flux_Kt,flux_err_Kt,flux_Kt_uncorrected,flux_err_Kt_uncorrected,flux_W1t,flux_err_W1t,flux_W1t_uncorrected,flux_err_W1t_uncorrected,flux_W2t,flux_err_W2t,flux_W2t_uncorrected,flux_err_W2t_uncorrected,flux_uc,flux_err_uc,flux_gc,flux_err_gc,flux_rc,flux_err_rc,flux_ic,flux_err_ic,flux_Zc,flux_err_Zc,flux_Yc,flux_err_Yc,flux_Jc,flux_err_Jc,flux_Hc,flux_err_Hc,flux_Kc,flux_err_Kc,flux_W1c,flux_err_W1c,flux_W2c,flux_err_W2c,mask,starmask,starscol,starssize,class,noOPT_r,noOPT_i,noIR_Z,noIR_Y,duplicate,mag_Zc,mag_Zt_fake,cluster_label,cluster_label_no_u,umap1_no_u,umap1_u,umap2_no_u,umap2_u,umap3_no_u,umap3_u,umap4_no_u,umap4_u,umap5_no_u,umap5_u,umap6_no_u,umap6_u,umap7_no_u,umap7_u,umap8_no_u,umap8_u,umap9_no_u,umap9_u,umap10_no_u,umap10_u,final_label,umap1,umap2,umap3,umap4,umap5,umap6,umap7,umap8,umap9,umap10,missing_bands
uberID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
36971195800254,waves_postprocessed_37.0_-3.5.rds,3697.0,28,11957.5,253.5,8276.934819,36.585878,-4.062106,36.585862,-4.062085,36.585896,36.585896,-4.062111,-4.062111,36.585931,-4.062128,36.585918,-4.062081,36.585862,-4.062085,36.585897,-4.062102,36.585862,-4.062085,36.585880,-4.062105,36.585862,-4.062085,-0.002751,0.003172,-0.162571,-0.094759,-0.164494,-0.161807,-0.153093,15.489770,0.033257,0.670359,0.980934,0.849038,0.661603,0.601284,0.565228,2.173057,11.660159,4442,0.935971,73.598735,28,10,16.045604,15.467905,18.671152,17.160600,16.335561,15.793864,3.380947e-05,2.207446e-06,2.917651e-05,1.904956e-06,2.949221e-04,8.025781e-07,2.636068e-04,7.173589e-07,1.028585e-03,9.746135e-07,9.482575e-04,8.985013e-07,1.729963e-03,3.315048e-06,1.623443e-03,3.110929e-06,0.002360,2.875208e-06,0.002250,2.741703e-06,0.002961,6.667548e-06,0.002853,6.424377e-06,0.002982,3.570833e-06,0.002903,3.476267e-06,0.003652,3.577042e-06,0.003589,3.515583e-06,0.002888,5.177355e-06,0.002855,5.119185e-06,1.286970e-03,8.837989e-07,1.279061e-03,8.783678e-07,6.698090e-04,3.140175e-06,6.669475e-04,3.126760e-06,2.929688e-05,1.597023e-06,2.939962e-04,5.770235e-07,1.022830e-03,6.629288e-07,1.721307e-03,1.980520e-06,2.350237e-03,1.182835e-06,2.947524e-03,2.627846e-06,0.002970,1.530534e-06,3.634016e-03,1.619906e-06,0.002868,2.629908e-06,1.218787e-03,3.587485e-07,6.390815e-04,1.514522e-06,0.0,0.0,3.0,3.0,star,0.0,0.0,0.0,0.0,0.0,15.472221,0,star,star,9.401932,7.795445,6.094347,2.986492,0.191821,-0.074320,7.363220,6.851164,4.796186,4.194026,4.850208,5.541643,6.239841,6.277967,5.597840,7.013787,6.083695,5.698566,4.561788,5.498656,star,7.795445,2.986492,-0.074320,6.851164,4.194026,5.541643,6.277967,7.013787,5.698566,5.498656,False
36971349600916,waves_postprocessed_37.0_-3.5.rds,3697.0,199,13495.5,915.5,9009.575018,36.457390,-4.006810,36.457419,-4.006849,36.457371,36.457454,-4.006875,-4.006875,36.457419,-4.006844,36.457410,-4.006829,36.457419,-4.006849,36.457384,-4.006818,36.457336,-4.006849,36.457388,-4.006811,3

In [24]:
df['uberID']=df.index

In [25]:
# Saving
df.to_csv('../../data/WD02/WD02_UMAP_full.csv')

In [26]:
columns_to_save = ['uberID', 'RAmax', 'Decmax', 'mag_Zt','final_label', 'missing_bands', 'umap1', 'umap2', 'umap3', 'umap4', 'umap5', 'umap6', 'umap7', 'umap8', 'umap9', 'umap10', 'class']

In [27]:
df[columns_to_save].to_csv('../../data/WD02/WD02_UMAP_reduced_columns.csv')